In [ ]:
from importlib import reload
import os

from obspy.clients.filesystem.sds import Client as SDSClient
from obspy.clients.fdsn import RoutingClient, Client
from obspy.core import UTCDateTime as UTC
from obspy import read_inventory
#from obspy.signal import util

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

from data_quality_control import sds_db, base, util, analysis

## Define parameters

In [ ]:
nslc_code = "GR.BFO..BHZ"

overlap = 60 #3600
fmin, fmax = (4, 14)
nperseg = 2048
winlen_in_s = 3600
proclen = 24*3600

outdir = 'output'

sds_root = os.path.abspath('../../sample_sds/')
#inventory_routing_type = "eida-routing"
inventory_routing_type = read_inventory("../../sample_sds/GR.BFO..BHZ.xml")

In [ ]:
startdate = UTC("2020-12-24")
enddate = UTC("2021-01-15")

In [ ]:
sdsclient = SDSClient(sds_root)

if isinstance(inventory_routing_type, str):
    invclient = RoutingClient(inventory_routing_type)
else:
    invclient = inventory_routing_type
sdsclient.get_all_nslc()

# Times in data

In [ ]:
startdate.julday

In [ ]:
sdsclient.get_waveforms(*nslc_code.split("."), a
                       )

# NSCProcessor

In [ ]:
reload(base)
nscproc = base.NSCProcessor(*nslc_code.split("."), sdsclient, invclient )

output = nscproc.process(startdate, enddate)

In [ ]:
plt.imshow(np.log10(output.psds.T), aspect="auto")

In [ ]:
plt.plot(output.amplitudes)

In [ ]:
output

In [ ]:
output.trim_nan()

In [ ]:
output

# SDSProcessor

i.e. processing of multiple years, writing to files

In [ ]:
startdate

In [ ]:
reload(sds_db)
reload(base)
proc_params = base.ProcessingParameters(proclen_seconds=proclen)
sdsp = sds_db.SDSProcessor(nslc_code,
                                    inventory_routing_type, sds_root, 
                                        outdir=outdir, **proc_params.get_dict())
#print(sdsp)

sdsp.process(UTC("2020-12-20"), UTC("2020-12-31"))

In [ ]:
(UTC("2021-01-10T22:00:00.000000") - UTC("2021-01-01T00:00:00.000000Z")) / 3600

In [ ]:
215-192

In [ ]:
sdsp.endtime

In [ ]:
reload(base)
nscproc = base.NSCProcessor(*nslc_code.split("."), sdsclient, inventory_routing_type )

output = nscproc.process(UTC("2021-01-01"), enddate)

In [ ]:
output

In [ ]:
output.psds.shape

# Test assertion in BaseProcessedData

We manipulate the data arrays so that they contain one value/row more than expected from the set times.

In [ ]:
output.psds = np.append(output.psds, output.psds[-1,:][None,:], axis=0)
output.amplitudes = np.append(output.amplitudes, 0)

In [ ]:
output.amplitudes.shape, output.psds.shape

In [ ]:
output._check_shape_vs_time()

# Extend output objects

In [ ]:
reload(base)
nscproc = base.NSCProcessor(*nslc_code.split("."), sdsclient, inventory_routing_type )

output = nscproc.process(UTC("2020-12-20"), UTC("2020-12-31"))
output += nscproc.process(UTC("2021-01-01"), enddate)

# Plotting

In [ ]:
reload(base)
nscproc = base.NSCProcessor(*nslc_code.split("."), sdsclient, inventory_routing_type )

output = nscproc.process(UTC("2020-12-20"), UTC("2020-12-31"))
output += nscproc.process(UTC("2021-01-01"), enddate)

In [ ]:
print(output)

In [ ]:
output.plot_amplitudes()

In [ ]:
print(output)

In [ ]:
output.plot_psds(np.log10)

## Reshaping of flat array

In [ ]:
X = np.array([np.arange(24) for i in range(16)]).ravel()

In [ ]:
X.reshape(16, 24)

In [ ]:
plt.imshow(X.reshape(16, 24).T, aspect="auto")

# Analysis module

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
reload(analysis)
reload(base)
lyza = analysis.Analyzer(outdir, nslc_code)

lyza.get_data(startdate, enddate)

In [ ]:
lyza.plot_spectrogram();

In [ ]:
amp3d, psd3d = lyza.plot3d()

In [ ]:
display(HTML(amp3d.to_html()))

In [ ]:
display(HTML(psd3d.to_html()))

# New Analzyer

In [ ]:
reload(analysis)
lyza = analysis.Analyzer2(outdir, nslc_code)

In [ ]:
lyza.get_available_datafiles()

In [ ]:
lyza.get_available_timerange()

In [ ]:
lyza.get_data(startdate, enddate)

In [ ]:
lyza

In [ ]:
lyza.plot_amplitudes()